In [17]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [18]:
electricity = pd.read_excel('../input/Buildings_el.xlsx', sheet_name='Electricity kWh', header=1, index_col=0)
weather = pd.read_excel('../input/Buildings_el.xlsx', sheet_name='Weather archive', header=2, index_col=0)
areas = pd.read_excel('../input/Buildings_el.xlsx', sheet_name='Areas')

In [19]:
print(f"Electricity shape: {electricity.shape}")
print(f"Weather shape: {weather.shape}")
print(f"Areas shape: {areas.shape}")

Electricity shape: (8759, 10)
Weather shape: (17536, 12)
Areas shape: (10, 2)


In [20]:
electricity.shape

(8759, 10)

In [21]:
electricity

,ICT,"U06, U06A, U05B",OBS,"U05, U04, U04B, GEO",TEG,LIB,MEK,SOC,S01,D04
Timestamp,,,,,,,,,,
2023-01-01 00:00:00,135.0,98.4,7.540,84.9,5,27.2,13.6,67.5,2.9,10.6
2023-01-01 01:00:00,134.7,96.4,7.299,85.8,6,27.0,13.7,67.2,2.9,12.3
2023-01-01 02:00:00,133.1,98.1,7.129,84.8,5,27.0,13.8,67.4,2.8,12.0
2023-01-01 03:00:00,129.9,96.3,7.258,84.9,6,26.2,13.7,67.3,2.8,10.9
2023-01-01 04:00:00,130.7,97.4,7.238,94.1,5,26.8,14.0,67.6,2.9,11.5
...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,121.9,104.5,7.830,86.4,5,38.8,12.4,77.2,17.2,20.2
2023-12-31 20:00:00,121.1,103.4,7.836,81.1,5,38.5,11.1,75.3,17.6,21.1
2023-12-31 21:00:00,132.5,100.3,7.744,80.1,5,37.4,10.6,74.6,17.0,18.2


In [22]:
expected_range = pd.date_range(start='2023-01-01 00:00', end='2023-12-31 23:00', freq='h')
actual_range = electricity.index

missing = expected_range.difference(actual_range)

print("Missing timestamp(s):")
print(missing)
print(f"Missing timestamps: {len(missing)}")

Missing timestamp(s):
DatetimeIndex(['2023-03-26 03:00:00', '2023-03-26 04:00:00'], dtype='datetime64[ns]', freq='h')
Missing timestamps: 2


In [23]:
electricity = electricity.groupby(electricity.index).mean()
electricity = electricity.reindex(expected_range)

In [24]:
dupes = electricity.index[electricity.index.duplicated()]
print("Duplicate timestamps:", dupes)
print("Count of duplicates:", len(dupes))

Duplicate timestamps: DatetimeIndex([], dtype='datetime64[ns]', freq='h')
Count of duplicates: 0


In [25]:
electricity.interpolate(method='linear', limit_direction='both', inplace=True)

In [26]:
electricity.to_pickle('../input/electricity_only.pkl')
weather.to_pickle('../input/weather_only.pkl')
areas.to_pickle('../input/areas_only.pkl')

In [27]:
df = pd.DataFrame(columns=['Timestamp', 'Building_ID', 'Usage'])

start = datetime(2023, 1, 1)
end = datetime(2024, 1, 1)
date_range = pd.date_range(start, end, freq='h')[:-1]

for building in tqdm(electricity.columns.values):
    df_building = pd.DataFrame(columns=['Timestamp', 'Building_ID', 'Usage'])
    df_building['Timestamp'] = date_range
    df_building['Building_ID'] = building
    df_building['Usage'] = electricity[building].values
    df = pd.concat([df, df_building]).reset_index(drop=True)

df

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2208\2282367637.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_building]).reset_index(drop=True)
100%|██████████| 10/10 [00:00<00:00, 317.45it/s]


,Timestamp,Building_ID,Usage
0,2023-01-01 00:00:00,ICT,135.0
1,2023-01-01 01:00:00,ICT,134.7
2,2023-01-01 02:00:00,ICT,133.1
3,2023-01-01 03:00:00,ICT,129.9
4,2023-01-01 04:00:00,ICT,130.7
...,...,...,...
87595,2023-12-31 19:00:00,D04,20.2
87596,2023-12-31 20:00:00,D04,21.1
87597,2023-12-31 21:00:00,D04,18.2
87598,2023-12-31 22:00:00,D04,18.1


In [28]:
df["Month"] = df["Timestamp"].dt.month
df["Day_of_month"] = df["Timestamp"].dt.day
df["Day_of_week"] = df["Timestamp"].dt.dayofweek
df["Hour"] = df["Timestamp"].dt.hour
df

,Timestamp,Building_ID,Usage,Month,Day_of_month,Day_of_week,Hour
0,2023-01-01 00:00:00,ICT,135.0,1,1,6,0
1,2023-01-01 01:00:00,ICT,134.7,1,1,6,1
2,2023-01-01 02:00:00,ICT,133.1,1,1,6,2
3,2023-01-01 03:00:00,ICT,129.9,1,1,6,3
4,2023-01-01 04:00:00,ICT,130.7,1,1,6,4
...,...,...,...,...,...,...,...
87595,2023-12-31 19:00:00,D04,20.2,12,31,6,19
87596,2023-12-31 20:00:00,D04,21.1,12,31,6,20
87597,2023-12-31 21:00:00,D04,18.2,12,31,6,21
87598,2023-12-31 22:00:00,D04,18.1,12,31,6,22


In [29]:
df.reset_index(drop=True, inplace=True)
df

,Timestamp,Building_ID,Usage,Month,Day_of_month,Day_of_week,Hour
0,2023-01-01 00:00:00,ICT,135.0,1,1,6,0
1,2023-01-01 01:00:00,ICT,134.7,1,1,6,1
2,2023-01-01 02:00:00,ICT,133.1,1,1,6,2
3,2023-01-01 03:00:00,ICT,129.9,1,1,6,3
4,2023-01-01 04:00:00,ICT,130.7,1,1,6,4
...,...,...,...,...,...,...,...
87595,2023-12-31 19:00:00,D04,20.2,12,31,6,19
87596,2023-12-31 20:00:00,D04,21.1,12,31,6,20
87597,2023-12-31 21:00:00,D04,18.2,12,31,6,21
87598,2023-12-31 22:00:00,D04,18.1,12,31,6,22


In [30]:
areas

,Buid_ID,Area [m2]
0,ICT,11413.0
1,"U06, U06A, U05B",15413.0
2,OBS,160.4
3,"U05, U04, U04B, GEO",13654.0
4,TEG,1968.1
5,LIB,11062.0
6,MEK,4434.0
7,SOC,10360.0
8,S01,5302.0
9,D04,4323.6


In [31]:
areas.rename(columns={"Buid_ID": "Building_ID"}, inplace=True)
df_new = df.merge(areas, on="Building_ID", how="left")
df_new

,Timestamp,Building_ID,Usage,Month,Day_of_month,Day_of_week,Hour,Area [m2]
0,2023-01-01 00:00:00,ICT,135.0,1,1,6,0,11413.0
1,2023-01-01 01:00:00,ICT,134.7,1,1,6,1,11413.0
2,2023-01-01 02:00:00,ICT,133.1,1,1,6,2,11413.0
3,2023-01-01 03:00:00,ICT,129.9,1,1,6,3,11413.0
4,2023-01-01 04:00:00,ICT,130.7,1,1,6,4,11413.0
...,...,...,...,...,...,...,...,...
87595,2023-12-31 19:00:00,D04,20.2,12,31,6,19,4323.6
87596,2023-12-31 20:00:00,D04,21.1,12,31,6,20,4323.6
87597,2023-12-31 21:00:00,D04,18.2,12,31,6,21,4323.6
87598,2023-12-31 22:00:00,D04,18.1,12,31,6,22,4323.6


In [32]:
df_new.to_pickle('../input/data.pkl')